## Deploy Notebook Function

by: Alex Comerford (alexanderjcomerford@gmail.com)

In this notebook we will guide the user through the steps to deploying a jupyter notebook as a serverless function on openfaas.

### Setup

First we will setup the openfaas cli and environment variables to allow us to directly talk to openfaas.

In [1]:
## Install the openfaas cli
!curl -sL https://cli.openfaas.com | sh

## Setup openfaas url for deployment    
%env OPENFAAS_URL=gateway-external.openfaas.svc.cluster.local:8080

x86_64
Download complete.

Running as root - Attempting to move faas-cli to /usr/local/bin
New version of faas-cli installed to /usr/local/bin
Creating alias 'faas' for 'faas-cli'.
  ___                   _____           ____
 / _ \ _ __   ___ _ __ |  ___|_ _  __ _/ ___|
| | | | '_ \ / _ \ '_ \| |_ / _` |/ _` \___ \
| |_| | |_) |  __/ | | |  _| (_| | (_| |___) |
 \___/| .__/ \___|_| |_|_|  \__,_|\__,_|____/
      |_|

CLI:
 commit:  5b36c50280d961d3aa248bd17f901f4ef2774447
 version: 0.8.1
env: OPENFAAS_URL=gateway-external.openfaas.svc.cluster.local:8080


### Deploying a notebook

There are only two requirements to deploy a notebook as a function in openfaas

1. Write a jupyter notebook 
2. Deploy the 

In [2]:
NOTEBOOK_TO_DEPLOY = "./notebook_string_test.ipynb"

In [85]:
import os

funcname=os.path.splitext(
    os.path.basename(NOTEBOOK_TO_DEPLOY))[0].lower().replace("_","-")

!./faas-cli build --image={funcname} \
                  --handler=./       \
                  --name={funcname}  \
                  --lang=dockerfile  \
                  --no-cache         \
                  --build-arg notebook={NOTEBOOK_TO_DEPLOY}

!./faas-cli deploy --image={funcname} \
                   --name={funcname}

Building: notebook-string-test:latest with Dockerfile. Please wait..
Sending build context to Docker daemon   8.04MB
Step 1/14 : FROM serverless_datascience:base
 ---> a2fae016518c
Step 2/14 : ENV PLACEHOLDER_NOTEBOOK=/notebook.ipynb
 ---> Running in 75cff523c4a7
Removing intermediate container 75cff523c4a7
 ---> e4bb94663c63
Step 3/14 : ENV OUTPUT_NOTEBOOK=/out.ipynb
 ---> Running in 15aa2a32a443
Removing intermediate container 15aa2a32a443
 ---> 971bfab3f661
Step 4/14 : ENV JQ_NOTEBOOK_STDOUT="[.cells | .[] | .outputs] | del(.[] | select( (. == []) or (. == null)))[0] | [ .[] | select( .name == \"stdout\" ) ][0] | .text[0]"
 ---> Running in e7650e53952f
Removing intermediate container e7650e53952f
 ---> ee37969f5334
Step 5/14 : ARG notebook
 ---> Running in 733648d96711
Removing intermediate container 733648d96711
 ---> 12bf7e64bc19
Step 6/14 : ENV INPUT_NOTEBOOK=$notebook
 ---> Running in e67976d7d093
Removing intermediate container e67976d7d093
 ---> 2b142b9b9906
Step 7/14 : ADD $I

In [90]:
!curl --data '{"TEST": "hello world","DATA": [9,9,9]}' \
      -H "Content-Type: application/json" \
      -H "Accept: text/plain" \
      gateway-external.openfaas.svc.cluster.local:8080/function/notebook-string-test

   0
0  9
1  9
2  9
